# PLIP — Protein ligand interaction profiling

See [the tutorial](/Tutorials/protein-ligand-interactions.ipynb).

In [1]:
# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/exess_rex/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)

In [2]:
# Get a pdb to work with
# We use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch '1MBN' > '1MBN.pdb'

In [3]:
from pathlib import Path

import rush

client = rush.build_blocking_provider_with_functions(batch_tags=["exess_rex"])
o = client.exess_rex(
      {
    "driver": "Optimization",
    "model": {
      "method": "RestrictedHF",
      "basis": "STO-3G",
      "standard_orientation": "None"
    },
    "system": {
      "max_gpu_memory_mb": 1000
    },
    "keywords": {
      "scf": {
        "use_ri": False,
        "max_iters": 100,
        "max_diis_history_length": 8,
        "convergence_threshold": 1e-10,
        "convergence_metric": "DIIS"
      },
      "optimization": {
        "max_iters": 200,
        "convergence_criteria": {
          "metric": "Baker",
          "gradient_threshold": 5.66918e-4,
          "delta_energy_threshold": 1e-6,
          "step_component_threshold": 1.2e-3
        }
      },
      "export": {
        "export_bond_orders": True,
        "export_gradient": True
      }
    },
    "schema_version": "0.2.0"
  },
  [Path("/home/machineer/Programming/tengu-hermes/tests/exess_basic/input_topology.json")],
  resources={"gpus": 1, "storage": 100, "storage_units": "MB", "walltime": 60}, target="BULLET"
)

In [4]:
o

(Arg(id=0d92aa69-ec20-4804-98e2-592ea2fae6d2, value=None),)

In [5]:
help(client.exess_rex)

Help on function exess_rex in module rush.provider:

exess_rex(*args: *tuple[Record, list[RushObject[Record]]], target: 'Target | None' = None, resources: 'Resources | None' = None, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[typing.Tuple[Optional[None], Optional[str]]]
    Runs an arbitrary EXESS calculation, provided with valid parameters as described by the input.md document

    Module version:
    `github:talo/tengu-exess/bf963c302854d00dfacf11ea592f0ddbef1a4f50#exess_rex`

    QDX Type Description:

        in: ExessParams {
            keywords: Keywords {
                force_field: ForceFieldKeywords {ff_filename: string}?,
                rtat: RTATKeywords {
                    enabled: bool?,
                    json_file_dump_prefix: string?,
                    synchronous: bool?
                }?,
                boundary: BoundaryKeywords {
                    y: BoundaryCondition {
                        kind: BoundaryConditionKind[Perio

In [6]:
o[0].get()

2024-12-26 14:33:14,415 - rush - INFO - Argument 0d92aa69-ec20-4804-98e2-592ea2fae6d2 is now ModuleInstanceStatus.ADMITTED
2024-12-26 14:33:28,546 - rush - INFO - Argument 0d92aa69-ec20-4804-98e2-592ea2fae6d2 is now ModuleInstanceStatus.DISPATCHED
2024-12-26 14:33:34,256 - rush - INFO - Argument 0d92aa69-ec20-4804-98e2-592ea2fae6d2 is now ModuleInstanceStatus.AWAITING_UPLOAD


{'Ok': {'path': 'e08d8a3d-4453-420e-92ae-c4c45f537663',
  'size': 0,
  'format': 'json'}}

In [7]:
o[0].download()

PosixPath('objects/e08d8a3d-4453-420e-92ae-c4c45f537663')

In [8]:
o[0].typeinfo

{'k': 'fallible',
 't': [{'k': 'record',
   't': {'path': {'k': '@',
     't': {'doc': {'fields': {'trajectory_qmmbes': 'The trajectory of the QM/MMBE energy for each fragment.\nThe `i`th index corresponds to the `i`th frame of a `Dynamics` calculation. Not present in non-`Dynamics` calculations',
        'trajectory': 'The trajectory of the QM/MMBE energy.',
        'qmmbe': 'The QM/MMBE energy.',
        'calculation_time': 'The time it took to run the calculation.'}},
      'k': 'record',
      't': {'trajectory': {'k': 'optional',
        't': {'doc': {'fields': {}},
         'k': 'record',
         't': {'frames': {'k': 'array',
           't': {'doc': {'fields': {}},
            'k': 'record',
            't': {'estimated_temperature': 'f64',
             'velocities': {'k': 'optional', 't': {'k': 'array', 't': 'f64'}},
             'parent': {'k': 'optional',
              't': {'k': 'record', 't': ['u32'], 'n': 'FrameRef'}},
             'simulation_time': 'f64',
             '